In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, merge, UpSampling2D
from keras.layers.merge import concatenate

import cv2
from tqdm import tqdm


Using Theano backend.


In [4]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = all_info[0]
x_test = all_info[1]
y_train = all_info[2]
y_test = all_info[3]

In [19]:
def get_unet():
    inputs1 = Input(shape=(32, 32, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs1)
    conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv12)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv23 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv23)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv32 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv32)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv42 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv42)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv52 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv52), conv42], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv62 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv62), conv32], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv72 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv72), conv23], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv82 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv82), conv12], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv92 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    flat = Flatten()(conv92)
    dense1 = Dense(128, activation='relu')(flat)
    drop = Dropout(0.5)(dense1)
    dense2 = Dense(17, activation='sigmoid')(drop)

    model = Model(inputs=inputs1, outputs=dense2)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(35000, 32, 32, 4)
(5479, 32, 32, 4)
(35000, 17)
(5479, 17)


In [20]:
print (k.__version__)
model = get_unet()

2.0.4


In [7]:
from keras.models import load_model

In [11]:
model.load_weights('models/unets/unet_baseline_weights.h5')

In [16]:
x_test = x_test[:,:,:,:3]
print(x_test.shape)
print(y_test.shape)

(5479, 32, 32, 3)
(5479, 17)


In [21]:
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))
model.evaluate(x_test, y_test, batch_size=128, verbose=1)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]
[[ 0.22830021  0.29662803  0.72978938 ...,  0.79081875  0.83516175
   0.41232634]
 [ 0.17382061  0.24228986  0.78558916 ...,  0.85976648  0.8899408
   0.38880515]
 [ 0.16934729  0.23150642  0.78950453 ...,  0.8408013   0.89182633
   0.36658847]
 ..., 
 [ 0.09311847  0.18027595  0.85442156 ...,  0.92160785  0.94412118
   0.29726776]
 [ 0.12030565  0.1572516   0.87774575 ...,  0.90235341  0.93766409
   0.37094408]
 [ 0.12998934  0.19770357  0.7804141  ...,  0.88400233  0.91325623
   0.28661466]]
0.21584261652
5479/5479 [==============================] - 107s   


[1.0993399612005881, 0.52225073108622622]